##HW 5.4 (2) Pairwise Distances
The step produces the following output:  
["artists", "whilst"]	[2567.9386674918856, 16197.0]  
["artists", "wipe"]	[3125.7741441121429, 17348.0]  
["artists", "worshipped"]	[3058.6982525250837, 14961.0]  
["artists", "year's"]	[2680.9888101221163, 17719.0]  
["arts", "abandonment"]	[10995.817386624789, 48042.0]  
["arts", "abroad"]	[11316.550799603208, 48720.0]  
The output is stored at s3://ucb-mids-mls-vineetgangwar/HW54/2
  
####Code Description
1. The job calculates Euclidean and Manhattan distance
2. The mapper_init function reads all the top 10K words generated in the step 5.4.1.b above
3. The mapper reads in each of the 10K words along with the co-occurrence vectors one by one
4. The mapper then creates combinations of pairs from the 10K words read in step 2 above
5. For each input word,vector the mapper emits pair,vector for all the pairs in step 4 above that contains the word
e.g.  
if input to mapper is 'abstract' 2,4,5,2,4,...  
then the mapper will output the vector along with any pair that has 'abstract' in it  
('the', 'abstract') 2,4,5,2,4,...  
('abstract', 'end;) 2,4,5,2,4,...  
5. The reducer then receives each pair along with a list of two co-occurrence vectors. One vector for each word in the pair
6. The reducer then calculates the distances and outputs each pair along with Euclidean and manhattan distances

In [ ]:
%%writefile hw542.py


# Calculating distances

from mrjob.job import MRJob
from mrjob.step import MRStep
import numpy as np
import urllib2
import itertools

def euclidean_distance(np_data1, np_data2):
    # Calculate Euclidean distance between two points in 10000 dimensional space
    diff = np_data1 - np_data2
    diff_sq = diff**2
    diff_sq_sum = sum(diff_sq)
    dist = diff_sq_sum**0.5
    return dist

def manhattan_distance(np_data1, np_data2):
    # Calculate Manhattan distance between two points in 10000 dimensional space
    return np.absolute(np_data1 - np_data2).sum()

class hw542(MRJob):
    def mapper_init(self):
        self.wordlist = list()
        
        # Reading in 10K words
        s3_path = 'https://s3.amazonaws.com/ucb-mids-mls-vineetgangwar/HW54/1b/part-00000'
        filehndl = urllib2.urlopen(s3_path)
        
        for line in filehndl:
            word = line.split()[1].replace('"', '')
            self.wordlist.append(word)
        filehndl.close()
    
    def mapper(self, line_no, line):
        [word, vector] = line.strip().split()
        word = word.replace('"', '')
        vector = vector.replace('"', '')[:-1]

        self.pairs = itertools.combinations(self.wordlist, 2)
        for pair in self.pairs:
            if word in pair:
                yield pair, vector
                
    def reducer(self, pair, vectors):
        [v1, v2] = vectors
        v1 = v1.split(',')
        v1 = np.array(v1).astype(float)
        v2 = v2.split(',')
        v2 = np.array(v2).astype(float)
        e_dist = euclidean_distance(v1, v2)
        m_dist = manhattan_distance(v1, v2)
        yield pair, (e_dist, m_dist)

    def steps(self):
        return [
            MRStep(mapper_init    = self.mapper_init,
                   mapper         = self.mapper,
                   reducer        = self.reducer)
        ]

if __name__ == '__main__':
    hw542.run()

In [ ]:
!python hw542.py -r emr s3://ucb-mids-mls-vineetgangwar/HW54/1c/ --output-dir=s3://ucb-mids-mls-vineetgangwar/HW54/2 --no-output